# Laserscanning - Exercise 4

#### Please upload the implemented solutions till <u>29.11.2022</u> to the studip folder of your group. The file should follow this format:
##### EX04_Group_XX.ipynb (e.g. EX04_Group_04.ipynb)
(In case you need to upload additional files, please also upload them using the same schema, e.g. EX04_Group_04.zip)

# Segmentation I

You will need the lecture script No. 5 “Segmentation I” (download: Stud.IP) to complete this assignment.

# Determination of plane parameters using PCA
Principal Component Analysis (PCA) calculates the eigenvector of a point cloud as the normal vector by finding out the parameters of the optimal plane fitting the provided data (lecture script No. 5, slides ~20-23). Implement the PCA as a function (input: point cloud as a $n\times3$ array) and estimate the corresponding plane(s) for all three provided point clouds (download: Stud.IP).

In [4]:
from plyfile import PlyData
import numpy as np

# load data
_data = PlyData.read('plane_tango.ply').elements[0].data
plane_tango = np.stack((_data['x'], _data['y'], _data['z']), axis=1)
_data = PlyData.read('plane_riegl1.ply').elements[0].data
plane_riegl1 = np.stack((_data['x'], _data['y'], _data['z']), axis=1) 
_data = PlyData.read('plane_riegl2.ply').elements[0].data
plane_riegl2 = np.stack((_data['x'], _data['y'], _data['z']), axis=1) 

print(plane_tango)
print(plane_riegl1)
print(plane_riegl2)

[[ 0.0693865  2.54306    0.0111053]
 [ 0.074412   2.54541    0.0109674]
 [ 0.0797195  2.54718    0.010857 ]
 ...
 [ 0.321962   2.79352   -0.0689245]
 [ 0.330892   2.80246   -0.0707128]
 [ 0.339948   2.81185   -0.0725622]]
[[ 0.369  2.778 -0.511]
 [ 0.369  2.776 -0.638]
 [ 0.37   2.786 -0.665]
 ...
 [-0.07   2.436 -0.248]
 [-0.07   2.441 -0.372]
 [-0.07   2.441 -0.545]]
[[ 1.393  2.717 -0.005]
 [ 1.393  2.716 -0.017]
 [ 1.381  2.706  0.034]
 ...
 [ 1.054  3.043 -0.505]
 [ 1.053  3.04  -0.511]
 [ 1.043  3.033 -0.387]]


In [5]:
# implement the PCA algorithm
def pca(points):
    ### BEGIN SOLUTION
    mean_points = np.mean(points,axis=0)
    #print(mean_points)
    list_mp =  mean_points.tolist()
    c_points = points - mean_points
    covariance_mat = c_points.T @ c_points  #covariance matrix
    eigen_vals, eigen_vecs = np.linalg.eig(covariance_mat)    
    #print(eigen_vals)
    list_eva = eigen_vals.tolist()
    lambda_ = min(eigen_vals)
    n = eigen_vecs[list_eva.index(lambda_)]  #the sought-for normal vector
    #print(n)
    list_n = n.tolist()
    #unit_n = n / np.linalg.norm(n)
    #print(np.linalg.norm(n))
    d = -(list_n[0]*list_mp[0]+list_n[1]*list_mp[1]+list_n[2]*list_mp[2])
    

    ### END SOLUTION
    
    #raise NotImplementedError()
    
    return lambda_ , n, d

In the following cell you can test your implementation and can compare it with the correct values for each of the three provided point cloud files. At some point the decimal values could slightly vary.

| file | lambda | normal vector | distance |
| --- | --- | --- | --- |
| plane_tango.ply | 0.297639 | [-0.69960505  0.71404618  0.02628372] | -1.75849068165 |
| plane_riegl1.ply | 0.859337 | [-0.67446429  0.73830175 -0.00290692] | -1.8286588192 |
| plane_riegl2.ply | 0.747634 | [-0.73383665 -0.67925513  0.00981276] | 2.85578155518 |

In [8]:
pca(plane_riegl2)

(0.74763775,
 array([-0.73383665, -0.67931044,  0.00459555], dtype=float32),
 2.8546816426362494)

<u>Please explain your code and the **PCA principle** here:</u>


Principal component analysis, or PCA, is a dimensionality-reduction method that is often used to reduce the dimensionality of large data sets, by transforming a large set of variables into a smaller one that still contains most of the information in the large set.

First we use np.mean() to find the center point of the points cloud and simplified coordinates. We transform the simplified coordinates and then dot itself to get the covariance matrix.  Then we use np.linalg.eig() to find his eigenvalues and eigenvectors.The sought-for normal vector n ist the eigenvector belonging to the smallest eigen value of symmetric matrix.The remaining parameter d is determined by d=- nTp=ax+by+cz(x,y,z are average values). Lambda is the smallest eigenvalue.

## Calculate the noise 𝜎 for every point cloud

Calculate the noise of the process using an **iterative approach** by calculating and summing the single $\nu_i$:

$$\sigma=\sqrt{\frac{\sum \nu_i^2}{m-3}}$$

In [9]:
def noise_it(points, n, d):
    ### BEGIN SOLUTION
    sum_v_i_square = 0
    for i in range(len(points)):
        v_i_square = (n[0]*points[i][0] + n[1]*points[i][1] + n[2]*points[i][2])**2
        sum_v_i_square += v_i_square
    sigma = sqrt(sum_v_i_square/(len(points)-3))

    ### END SOLUTION
    
    #raise NotImplementedError()
    
    return sigma

<u>Please explain your code and findings here:</u>


First we use a for loop to find sum_v_i_square, and then substituting into the formula we get sigma.

## Compare the noise to the smallest eigenvalue 𝜆 derived from the PCA

Calcultae the noise from the **smallest eigenvalue** and compare the results with the `noise_it` function. Hint: lecture script No.5 slide ~23.

$$\sigma=\sqrt{\frac{V^TV}{m-3}}=\sqrt{\frac{\lambda}{m-3}}$$

In [10]:
def noise_lambda(lambda_, m):
    ### BEGIN SOLUTION
    
    sigma = sqrt(lambda_/(m-3))
    
    ### END SOLUTION
    
    #raise NotImplementedError()
    
    return sigma    

<u>Please explain your code and findings here:</u>


Because vTv equals lambda_. So we directly substitute lambda_ and m into the equation to get sigma.

### 1) Compare the noise of the different planes to each other
### 2) How can the values be interpreted?
### 3) Where do possible differences arise from?

<u>Answer the above points here:</u>



The noise of the different planes to each other are different.
The values Standard deviation of the point distance can be interpreted by the formula shown above.We need to ask out vTV.
Here there are two ways to find the vTv. The first way we use Loop. we substitute each point into the plane equation, compute vi, compute square, sum up… The second way we use simpler solution. we note that n was obtained as eigenvector belonging to the smallest eigenvalue lambda of ATA. vTv is equal to lambda. Therefore, can be directly obtained from lambda.
Possible differences arise from lambda_ , n, d. The different planes have different lambda_ , n, d, so they noise are different. Some outlying points can have a big impact on the noise.

### Calculate the normal vectors and their intersection angles.
## Q: Are they perpendicular to each other?

In [11]:
### BEGIN SOLUTION
n_1=pca(plane_tango)[1]
n_2=pca(plane_riegl1)[1]
n_3=pca(plane_riegl2)[1]

Angle_1_2=np.arccos(np.dot(n_1,n_2)/(np.linalg.norm(n_1)*np.linalg.norm(n_2)))
print('plane_tango--plane_riegl1: ',Angle_1_2*(180/np.pi))
Angle_1_3=np.arccos(np.dot(n_1,n_3)/(np.linalg.norm(n_1)*np.linalg.norm(n_3)))
print('plane_tango--plane_riegl2: ',Angle_1_3*(180/np.pi))
Angle_2_3=np.arccos(np.dot(n_2,n_3)/(np.linalg.norm(n_2)*np.linalg.norm(n_3)))
print('plane_riegl1--plane_riegl2: ',Angle_2_3*(180/np.pi))

#Angle_1_2=np.arccos((np.dot(n_1[0],n_2[0])+np.dot(n_1[1],n_2[1])+np.dot(n_1[2],n_2[2]))/((np.sqrt(n_1[0] ** + n_1[1] ** + n_1[2] **))*(np.sqrt(n_2[0] ** + n_2[1] ** +n_2[2] **))))

### END SOLUTION

#raise NotImplementedError()

plane_tango--plane_riegl1:  10.041165259220588
plane_tango--plane_riegl2:  88.14713610719379
plane_riegl1--plane_riegl2:  90.22608176600559


<u>Please explain your code and and answer the question here:</u>

SOME SENTENCES

We have three points cloud and we can get three plane from those three points cloud,so i use the function pca and return the three normal vector n_1,n_2,n_3. And then i ues the function 'cosα=vector_a dot vector_b/norm_a*norm_b ' to calculate the intersection angles. If the angles equal to 90 dgree, this means the two vector perpendicular to each other and this means also the two plane perpendicular to each other.

From the results we can see, the plane_tango does not perpendicular to plane_riegl2 and the angle between plane_riegl1 and plane_riegl2 is very close to 90 dgree, so i think they perpendicular to each other.The angle between plane_tango and plane_riegl2 is 88.15 degree, from this result those two planes are not vertical.